### 資料準備


In [1]:
# 引入套件
import pandas as pd

In [ ]:
import openpyxl
# ram_df = pd.read_excel("./docs/0926_new.xlsx")

In [2]:
# 讀取 json 檔案包含轉換向量
df = pd.read_json("vector_all.json")

In [ ]:
df

### 轉換成文字雲


In [4]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import jieba

In [ ]:
# Generate a word cloud image
chinese_font_path = "jf-openhuninn-2.0.ttf"


def create_word_cloud(index, string):
    wordcloud = WordCloud(
        background_color="white",
        width=600,
        height=400,
        margin=2,
        font_path=chinese_font_path,
        prefer_horizontal=1,
        colormap="Set2",
    ).generate(string)
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    save_path = f"images/{index}.png"
    plt.savefig(save_path, dpi=300, bbox_inches='tight')

In [ ]:
df["jieba_cut"] = df["content"].apply(lambda x: " ".join(jieba.cut(x)))

In [ ]:
df.apply(lambda x: create_word_cloud(x.name, x["jieba_cut"]), axis=1)

In [32]:
for index, row in df.iterrows():
    if (index < 705):
        break
    print("Running:", index)
    stringtext = row["jieba_cut"]
    wordcloud = WordCloud(
        background_color="white",
        width=600,
        height=400,
        margin=2,
        font_path=chinese_font_path,
        prefer_horizontal=1,
        colormap="Set2",
    ).generate(stringtext)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    save_path = f"images/{index}.png"
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    # break

In [ ]:
print("done")

---


### Save Data


In [13]:
from pymongo import MongoClient

# 建立 MongoDB 連線
client = MongoClient("mongodb://100.90.12.119:27017/")
db = client.nthu_trello_helper
mongo_collection = db.article

In [5]:
from pymilvus import DataType, FieldSchema, CollectionSchema, Collection, connections
# 設定 Milvus 的連線資訊
connections.connect(alias="default")

# 設定 Milvus collection 名稱和維度
collection_name = 'trello_finder_tf'
# 設定分區名稱
partition_name = 'import'

In [6]:
milvus_collection = Collection(collection_name)

In [7]:
errorlist = []

In [20]:
# 將資料存入 MongoDB
for idx, row in df.iterrows():
    try:
        # 插入 MongoDB
        mongo_collection.insert_one({
            "link_id": str(idx),
            "title": row["title"],
            "url": row["url"],
            "content": row["content"],
            "importMilvus": True,
            "values": row["vector"],
            "class": row["class"],
            "tag": row["tag"],
            "tf": row["TF"],
        })

    except Exception as exp:
        print("MongoDB Insert Fail")
        print(exp)
        errorlist.append(idx)

In [8]:
# 將資料存入 Milvus
for idx, row in df.iterrows():
    try:

        insert_milvus_title = row["title"]
        if (len(insert_milvus_title) > 500):
            insert_milvus_title = insert_milvus_title[:497] + "..."

        try:
            # 插入 Milvus
            status = milvus_collection.insert({
                "id": str(idx),
                "track_id": str(idx),
                "value": row["vector_tf"],
                "title": insert_milvus_title,
            }, partition_name=partition_name)

            if (status.insert_count != 1):
                print("Milvus Insert Fail")
                print(status)
                errorlist.append(idx)
            # else:
                # print("Done \n--------")

        except Exception as exp:
            print("--------------------")
            print(str(idx), "|", len(str(idx)))
            print("Vector |", len(row["vector"]))
            print(insert_milvus_title, "|", len(insert_milvus_title))
            print("--------------------")
            print("Milvus Insert Fail")
            print(exp)
            errorlist.append(idx)

    except Exception as exp:
        print("MongoDB Insert Fail")
        print(exp)
        errorlist.append(idx)

RPC error: [insert_rows], <ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>, <Time:{'RPC start': '2023-08-02 20:57:01.029217', 'RPC error': '2023-08-02 20:57:01.029217'}>


--------------------
8370 | 4
Vector | 768
以假亂真，虛實難辨—虛擬實境 | 14
--------------------
Milvus Insert Fail
<ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>


RPC error: [insert_rows], <ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>, <Time:{'RPC start': '2023-08-02 20:57:01.248869', 'RPC error': '2023-08-02 20:57:01.248869'}>


--------------------
8432 | 4
Vector | 768
您所不知道的諾貝爾物理 | 11
--------------------
Milvus Insert Fail
<ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>


RPC error: [insert_rows], <ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>, <Time:{'RPC start': '2023-08-02 20:57:01.750078', 'RPC error': '2023-08-02 20:57:01.750078'}>
RPC error: [insert_rows], <ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>, <Time:{'RPC start': '2023-08-02 20:57:01.811520', 'RPC error': '2023-08-02 20:57:01.811520'}>
RPC error: [insert_rows], <ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>, <Time:{'RPC start': '2023-08-02 20:57:01.812520', 'RPC error': '2023-08-02 20:57:01.812520'}>
RPC error: [insert_rows], <ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>, <Time:{'RPC start': '2023-08-02 20:57:01.812520', 'RPC error': '2023-08-02 20:57:01.812520'}>
RPC error: [insert_rows], <ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>, <Time:{'RPC start': '2023-08-02 20:57:0

--------------------
8576 | 4
Vector | 768
看見微小世界（電子顯微鏡） | 13
--------------------
Milvus Insert Fail
<ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>
--------------------
8591 | 4
Vector | 768
『一家一枚 』科學海報：量子光學 | 16
--------------------
Milvus Insert Fail
<ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>
--------------------
8592 | 4
Vector | 768
什麼是暗物質與暗能量？ | 11
--------------------
Milvus Insert Fail
<ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>
--------------------
8593 | 4
Vector | 768
1901-1966年諾貝爾物理獎揭秘檔案 | 20
--------------------
Milvus Insert Fail
<ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>
--------------------
8594 | 4
Vector | 768
重力波大年表 | 6
--------------------
Milvus Insert Fail
<ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>


RPC error: [insert_rows], <ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>, <Time:{'RPC start': '2023-08-02 20:57:02.054212', 'RPC error': '2023-08-02 20:57:02.054212'}>


--------------------
8660 | 4
Vector | 768
科學女力：2018女物理人研討會紀實 | 18
--------------------
Milvus Insert Fail
<ParamError: (code=1, message=Collection field dim is 768, but entities field dim is 38)>


In [ ]:
errorlist

In [ ]:
for idx, row in df.iloc[errorlist].iterrows():
    print(idx)
    insert_milvus_title = row["title"]
    if (len(insert_milvus_title) > 75):
        insert_milvus_title = insert_milvus_title[:71] + "..."
    print(len(insert_milvus_title))
    print(row["title"])

In [ ]:
expr = 'id in ["21846"]'
milvus_collection.delete(expr)

### TF 標籤輸出 (相似文字搜尋)


In [9]:
def search_keyword(keyword):
    # Replace NaN values with empty strings
    df['TF'] = df['TF'].fillna('')
    # Search for the keyword in the TF column
    results = df[df['TF'].str.contains(keyword)]['TF'].tolist()

    # Split the results into individual words and count their occurrences
    word_counts = {}
    for result in results:
        words = result.split()
        for word in words:
            word_counts[word] = word_counts.get(word, 0) + 1

    # Sort the words based on their occurrence count in descending order
    sorted_words = sorted(word_counts.keys(),
                          key=lambda x: word_counts[x], reverse=True)

    # Return the top 6 words
    if (len(sorted_words) > 6):
        sorted_words = sorted_words[:6]
    return sorted_words

In [10]:
input_keyword = "藍光"
output = search_keyword(input_keyword)
print(output)

['藍光', 'LED', '中村', '藍色', '氮化', '睡眠']


In [11]:
def create_result_table(words_list):
    # 定義一個空的表格
    table_data = []
    # 定義一個空的錯誤列表
    errList = []
    # 逐一取出文字列表中的文字
    for word in words_list:
        try:
            # 將文字進行 search_keyword(文字) 並將結果存入 result
            result = search_keyword(word)
            # 將 word 和 result 存入表格中
            table_data.append({
                'word': word,
                'simliar_word': result,
            })
        except:
            # 如果有錯誤，將 word 存入錯誤列表中
            errList.append(word)
            continue

    # 將表格和錯誤列表回傳
    return {
        "data": table_data,
        "errList": errList,
    }


# 从 TF 列获取文字列表
words_list = df['TF'].str.split().explode().unique().tolist()

# 利用文字列表中的文字進行 search_keyword(文字) 并将结果输出成表格
result_table = create_result_table(words_list)
simliar_word_df = pd.DataFrame(result_table["data"]).fillna("")
# 將 simliar_word 列中的列表轉成字串
simliar_word_df["simliar_word_str"] = simliar_word_df["simliar_word"].apply(
    lambda x: " ".join(x))

In [15]:
from pymongo import MongoClient

# 建立 MongoDB 連線
client = MongoClient("mongodb://100.90.12.119:27017/")
db = client.nthu_trello_helper
mongo_collection = db.close_words

In [18]:
for idx, row in simliar_word_df.iterrows():
    # 插入 MongoDB
    mongo_collection.insert_one({
        "key": str(row['word']),
        "value": row['simliar_word_str']
    })